# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/)

In [1]:
import random
import time
import torch
import pandas as pd
from IPython.display import clear_output

import numpy as np
import tensorly
from tensorly import decomposition

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn
from torch.nn.functional import mse_loss

torch.manual_seed(0)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [2]:
def unfold(tensor, mode):
    return torch.reshape(torch.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1))

def fold(tensor, mode, shape):
    full_shape = list(shape)
    mode_shape = full_shape.pop(mode)
    full_shape.insert(0, mode_shape)
    return torch.moveaxis(torch.reshape(tensor, full_shape), 0, mode)

def my_composition(G_, U):
    data = G_.clone()
    for i in range(len(U)):
        init_shape = list(data.shape)
        new_shape = init_shape.copy()
        new_shape[i] = U[i].shape[0]
        G_i = unfold(data,i)
        data = fold((U[i]@G_i), i, new_shape);
    return data

In [3]:
def get_tensor(size=(100, 200, 300), r=10):
    U = [torch.randint(0,10,(s, r)).float() for s in size]
    G = torch.randint(0,10,(r, r, r)).float()
    data = my_composition(G, U)
    return data, U, G

In [4]:
size = (100,200,300)
r = 10

data, U, G = get_tensor(size, r)

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [5]:
noise = torch.rand(data.shape)/10
ndata = noise.add(data)

Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ: если тензор случайный, то он может не иметь разложения
и еще мы не сможем проверить ошибку восстановления ядра и матриц, если не будет делать их
шум - что-то вроде регуляризации

## 2 Реализуйте метод для восстановления тензора по разложению

## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [6]:
tG, tU= tensorly.decomposition.tucker(tensorly.tensor(ndata), G.shape)
tdata = torch.tensor(tensorly.tenalg.multi_mode_dot(tG, tU))
tdata.shape

torch.Size([100, 200, 300])

Не забудьте померить ошибку разложения по метрике MSE

In [7]:
print(f'mse_loss tucker lib = {torch.sum((tdata-ndata)**2)/torch.numel(tdata)}')

mse_loss tucker lib = 0.0010155184006124896


## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [8]:
import math
import torch
from torch.optim.optimizer import Optimizer

class Opt(Optimizer):

    def __init__(self, data, G, U, batch_size, lr=4e-4):
        self.data = data 
        self.batch_size = batch_size
        params = [G, *U]
        defaults = {"lr": lr}
        super().__init__(params, defaults)

    
    def step(self):
        grad = None
        while grad is None:
            l = len(self.param_groups[0]['params'])
            tensor = self.param_groups[0]['params'][np.random.randint(l)]
            grad = tensor.grad

        if not self.state:
            self.state["step"] = 1
        else:
            self.state["step"] += 1
            
        if self.state["step"] % 100 == 0:
            self.param_groups[0]['lr'] *= 1.005
            
        if self.state["step"] % 1000 == 0:
            self.param_groups[0]['lr'] *= 1.5
            
            
        elems = random.sample(range(1, tensor.numel()), self.batch_size)

        mask_flat = torch.zeros(tensor.numel())
        mask_flat[elems] = 1
        mask = mask_flat.reshape(tensor.shape)
        
        tensor.data.add_(grad * mask, alpha=-self.param_groups[0]['lr'] )

### 4.2 Реализуйте цикл оптимизации параметров

In [9]:
def training_loop(G, U, data, opt, itr, eps):
    loss_history = []
   
    for i in range(itr):
        loss = mse_loss(my_composition(G,U), data)
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        if i%400 == 0:
            print(f'itr {i} mse is {loss.data}')
            
        if loss < eps:
            break
            
        loss_history.append(loss.data.numpy())
        
    return loss_history

#### 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [10]:
size = (100,200,300)
r = 10
itr = 50000
lr = 1*10**-9
eps = 0.5
batch_size = 700

data, U, G = get_tensor(size, r)
noise = torch.rand(data.shape) / 10
data = noise.add(data)

G_opt = torch.randint(0, 10, (r, r, r)).float().requires_grad_()
U_opt = [torch.randint(0, 10, (s, r)).float().requires_grad_() for s in size]

my_optimizer = Opt(data, G_opt, U_opt, batch_size, lr)

In [11]:
start_time = time.time()

loss_history = training_loop(G_opt, U_opt, data, my_optimizer, itr, eps)

whole_time = time.time()-start_time

itr 0 mse is 42835152896.0


KeyboardInterrupt: 

In [ ]:
%%time
tG, tU= tensorly.decomposition.tucker(tensorly.tensor(data), G.shape)
tdata = torch.tensor(tensorly.tenalg.multi_mode_dot(tG, tU))
print(f'mse for lib optimizer: {mse_loss(tdata, data)}')

In [ ]:
data_opt = torch.tensor(repair_tensor(G_opt, U_opt))
print(f'mse for my optimizer: {mse_loss(data_opt, data)}')